# Joint Distributions of Q and $\tilde{E}_r$ to Compare to Edelweiss Paper

The purpose of this notebook is to detail the computation of the ER or NR bands with and arbitrary effective Fano factor applied. The construction used here is different from the one of Arvind, but it is verified to produce the same results for the probability distribution in ionization yield (called "Q" by Edelweiss) at a given fixed true recoil energy. 

